<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/NLP/2_Redes_Recurrentes/ejercicios/ejercicios.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg' /> </a>

# Ejercicios clase 2

# 1 - Análisis de sentimiento simple

En este práctico, crearemos un modelo de aprendizaje automático para detectar sentimientos (es decir, detectar si una oración es positiva o negativa) usando PyTorch y TorchText. Esto se hará en reseñas de películas, utilizando el [conjunto de datos de IMDb](http://ai.stanford.edu/~amaas/data/sentiment/).

Comenzaremos de manera muy simple para comprender los conceptos generales sin preocuparnos realmente por los buenos resultados. Las siguientes secciones se basarán en estos conceptos y obtendremos buenos resultados.



### Introducción

Usaremos una **red neuronal recurrente** (RNN), ya que se usan comúnmente en el análisis de secuencias. Un RNN toma una secuencia de palabras, $X = \{x_1, ..., x_T \}$, una a la vez, y produce un _estado oculto_, $h$, para cada palabra. Usamos el RNN _recurrentemente_ introduciendo la palabra actual $x_t$ así como el estado oculto de la palabra anterior, $h_{t-1}$, para producir el siguiente estado oculto, $h_t$.

$$h_t = \text{RNN}(x_t, h_{t-1})$$

Una vez que tenemos nuestro estado oculto final, $h_T$, (a partir de la última palabra en la secuencia, $x_T$) lo alimentamos a través de una capa lineal, $f$, (también conocida como capa densa), para recibir nuestro sentimiento predicho, $\hat{y} = f(h_T)$.

A continuación se muestra una oración de ejemplo, con la RNN prediciendo cero, lo que indica un sentimiento negativo. La RNN se muestra en naranja y la capa lineal se muestra en plateado. Tenga en cuenta que usamos la misma RNN para cada palabra, es decir, tiene los mismos parámetros. El estado oculto inicial, $ h_0 $, es un tensor inicializado a todos ceros.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment1.png?raw=1)

**Nota:** algunas capas y pasos se han omitido del diagrama, pero se explicarán más adelante.

Lo primero que haremos es dejar una semilla por defecto para los generadores de números aleatorios

In [ ]:
#Setup
import torch
import torchtext

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


## Preparando los datos


### Descarga de datos.

Lo primero que haremos será instalar el modulo `torchdata` que contiene algunos datasets comunes en deep learning. 

> NOTA: Luego de ejecutar esta celda es necesario reiniciar el entorno de Colab.

In [ ]:
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


A continuación descargaremos los datos de IMDB que usaremos para el modelo. `torchtext` tiene acceso a estos datos y los llamaremos con la clase correspondiente. Si bien la clase `IMDB` genera un `DataLoader`, vamos a hacer modificaciones para crear un `DataLoader` personalizado. Más adelante explicaremos nuestros argumentos. 

In [ ]:
import torchdata
import random
from torchtext.datasets import IMDB
train_data, test_data = IMDB(split=('train', 'test'))

# guardaremos momentaneamente los datos en listas.
full_list = list(train_data)
cutoff = int(len(full_list) * 0.7)

random.seed(SEED)
random.shuffle(full_list)

train_list = full_list[:cutoff] # entrenamiento
val_list = full_list[cutoff:] # validacion
test_list = list(test_data) # prueba


### Armado de vocabulario

Luego usaremos el tokenizador de spacy `en_core_web_sm` para crear la representación de las palabras de nuestro texto. Es decir, definiremos que un *token* es igual a una palabra.

In [ ]:
#@markdown # Ejercicio 1

# inserte su código



A continuación generaremos un vocabulario de `torchtext` como lo hicimos en clase. 

Para ello tenga en cuenta lo siguiente: 

* No incluiremos las palabras que se repitan menos de 10 veces. Para ello usaremos el argumento `min_freq` de la clase `vocab`.
* También añadimos 2 tokens adicionales: `<unk>` para palabras desconocidas y `<PAD>` como "relleno". El token de relleno nos permite que todas las oraciones tengan igual número de *tokens*. Para ello usaremos el argumento llamado `specials` de la clase `vocab` al que deberemos pasarle la siguiente tupla `('<unk>', '<PAD>')`

In [ ]:
#@markdown # Ejercicio 2
# inserte su codigo aqui
from collections import Counter
from torchtext.vocab import vocab


# inserte su codigo aqui
vocab = None #no cambie el nombre de esta variable

La siguiente linea le informa a vocab cual es el *token* que de usar al encontrar una palabra desconocida. Por eso incluimos ese token especial en la celda anterior 

In [ ]:
vocab.set_default_index(vocab['<unk>'])

Veamos algunos ejemplos de los métodos que ofrece la clase vocab

In [ ]:
print("La cantidad de tokens en el vocabulario es:", len(vocab))
s = 'the'
print(f"El índice del token '{s}' es {vocab[s]}")
itos = vocab.get_itos()
i = 2
print(f"El token con indice {i} es {itos[i]}")
s = "AYAYA"
print(f"A la palabra '{s}' le corresponde el token {itos[vocab[s]]}")

La cantidad de tokens en el vocabulario es: 18961
El índice del token 'the' es 10
El token con indice 2 es Not
A la palabra 'AYAYA' le corresponde el token <unk>


### Generación de `Dataloader`

El paso final para preparar los datos es crear los iteradores. Iteramos sobre estos en el ciclo de entrenamiento/evaluación, y devuelven un lote de ejemplos (indexados y convertidos en tensores) en cada iteración.

Usaremos una clase diseñada para devolver un lote de ejemplos con longitudes similares, minimizando la cantidad de relleno por ejemplo.

La nueva clase heredará de `Sampler` que es una clase de `torch`. `Sampler` se utiliza para indicarles a los `DataLoader` los índices que debe usar para armar cada minilote. Se necesita redefinir al menos 2 métodos: `__iter__` para armar un generador, `__len__` para saber la cantidad de elementos. Además crearemos un constructor `__init__`.

In [ ]:
import random
from torch.utils.data.sampler import Sampler

class BucketSampler(Sampler):

    def __init__(self, batch_size, train_list):
        self.length = len(train_list)
        self.train_list = train_list
        self.batch_size = batch_size
        indices = [(i, len(tokenizer(s[1]))) for i, s in enumerate(self.train_list)]
        random.seed(SEED)
        random.shuffle(indices)
        pooled_indices = []
        # creamos minilotes de tamaños similares
        for i in range(0, len(indices), self.batch_size * 100):
            pooled_indices.extend(sorted(indices[i:i + self.batch_size * 100], key=lambda x: x[1], reverse=True))

        self.pooled_indices = pooled_indices

    def __iter__(self):
        for i in range(0, len(self.pooled_indices), self.batch_size):
            yield [idx for idx, _ in self.pooled_indices[i:i + self.batch_size]]

    def __len__(self):
        return self.length
        return (self.length + self.batch_size - 1) // self.batch_size

Además tendremos definir una función `collate` que le indique al `DataLoader` los datos a incluir en cada minilote.

Para ello crearemos dos funciones anónimas. Una convertirá nuestro texto a números que podamos manejar en nuestras redes. La otra convertirá nestras etiquetas en números. Como solo tenemos 2 clases: reseña negativa y reseña positiva. Les asignaremos valores 0 y 1 respectivamente.

In [ ]:
text_transform = lambda x: [vocab[token] for token in tokenizer(x)]
label_transform = lambda x: 1 if x == 'pos' else 0

s = "here is an example"
print("Ejemplo de entrada para text_transform:", s)
print("Ejemplo de entrada para text_transform:", text_transform(s))



Ejemplo de entrada para text_transform: here is an example
Ejemplo de entrada para text_transform: [331, 44, 121, 1064]


In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list, length_list = [], [], []
    for (_label, _text) in batch:
        # convertimos la etiqueta usando label_transform
        label_list.append(label_transform(_label))
        # convertimos el texto en tokens
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)
        # guardamos la longitud de cada token
        length_list.append(len(processed_text))
    # armamos la tupla que conformara un ejemplo de minilote.
    result = (torch.tensor(label_list),
              pad_sequence(text_list, padding_value=1.0),
              torch.tensor(length_list) )
    return result

> NOTA: hemos incluido la longitud de la oración, porque es algo que necesitaremos en la segunda parte de nuestro trabajo práctico.

Ahora estamos listos para generar los `DataLoader` que necesitamos.

Para ello tenga encuenta que: 

* El `batch_size` deberá ser de 64 elementos
* Debera crear un `DataLoader` para entrenamiento, validación y prueba. También para cada uno deberá crear su instancia de `BucketSampler`
* En cada constructor deberá definir la función `collate` a utilizar con el atributo `collate_fn`.
* También deberá definir el *sampler* a utilizar con el atributo `batch_sampler`.

In [ ]:
#@markdown # Ejercicio 3

# inserte su código aquí
# no cambie los nombres de las siguientes variables
train_bucket = None
train_iter = None
val_bucket = None
val_iter = None
test_bucket = None
test_iter = None

## Construcción del modelo

### Arquitectura de nuestro modelo



La siguiente etapa es construir el modelo que eventualmente vamos a entrenar y evaluar.

Nuestra clase `Vanilla_RNN` es una subclase de` nn.Module`.

Dentro del `__init__` definimos las _capas_ del módulo. Nuestras tres capas son una capa _embedding_, nuestra RNN y una capa _densa_. Todas las capas tienen sus parámetros inicializados a valores aleatorios, a menos que se especifique explícitamente.

La capa de embedding se utiliza para transformar nuestro vector ralos one-hot (ralos ya que la mayoría de los elementos son 0) en un vector de embedding denso (denso ya que la dimensionalidad es mucho más pequeña y todos los elementos son números reales). Esta capa de embedding es simplemente una capa densa única. Además de reducir la dimensionalidad de la entrada al RNN, existe la teoría de que las palabras que tienen un impacto similar en el sentimiento de la revisión se mapean juntas en este denso espacio vectorial. Para obtener más información sobre los embeddings de palabras, consulte [aquí](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/).

La capa RNN es nuestra RNN que toma nuestro vector denso y el estado oculto anterior $h_{t-1}$, y lo usa para calcular el siguiente estado oculto, $ h_t $.

![](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment7.png?raw=1)

Finalmente, la capa linear toma el estado oculto final y lo alimenta a través de una capa densa, $ f(h_T) $, transformándola a la dimensión de salida correcta.

Se llama al método `forward` cuando introducimos ejemplos en nuestro modelo.

Cada lote, `text`, es un tensor de tamaño **[longitud de la oración, tamaño del lote]**. En un lote de oraciones, cada oración tiene sus palabras convertidas en un vector one-hot.

Puede notar que este tensor debería tener otra dimensión debido al tamaño de los vectores one-hot. Sin embargo PyTorch almacena convenientemente un vector one-hot como su valor de índice. Es decir, el tensor que representa una oración, es solo un tensor de los índices para cada token en esa oración. El acto de convertir una lista de tokens en una lista de índices se denomina comúnmente *numeralización*.

El lote de entrada luego se pasa a través de la capa de embedding lo que nos da una representación vectorial densa de nuestras oraciones. `embedded` es un tensor de tamaño **[longitud de la oración, tamaño del lote, embedding_dim]**.

`embedded` se introduce luego en la RNN. En algunos frameworks, debemos alimentar el estado oculto inicial, $ h_0 $, en la RNN. Sin embargo, en PyTorch, si no se pasa un estado oculto inicial como argumento, se establece de forma predeterminada en un tensor de todos ceros.

El RNN devuelve 2 tensores, `output` de tamaño **[longitud de la oración, tamaño de lote, dimensión de la variable oculta]** y` hidden` de tamaño **[1, tamaño de lote, dimensión de la variable oculta]**. `output` es la concatenación del estado oculto de cada paso de tiempo, mientras que ` hidden` es simplemente el estado oculto final. 

Finalmente, alimentamos el último estado oculto, `hidden`, a través de la capa lineal, `linear`, para producir una predicción.

### Implementación de nuestro modelo

A continuación, deberá crear la clase `Vanilla_RNN`.

Para ello tenga encuenta que debera:

* definir un constructor, `__init__` con tres capas:
  * Una instancia de `Embedding`
  * Una instancia de `RNN`
  * Una instancia de `Linear`

* Recuerde también que:
  * La dimensión de entrada de nuestra red es  la dimensión de los vectores one-hot. Es decir, es el tamaño del vocabulario.
  * La dimensión a la salida de `Embedding` es el tamaño de los vectores de palabras densas. Suele tener entre 50 y 250 dimensiones, pero depende del tamaño del vocabulario.
  * La dimensión del estado oculto es el tamaño de las variables ocultas. Suele rondar entre 100 y 500 dimensiones, pero también depende de factores como el tamaño del vocabulario, el tamaño de los vectores densos y la complejidad de la tarea.
  * La dimensión de salida suele ser el número de clases, sin embargo, en el caso de solo 2 clases, el valor de salida está entre 0 y 1 y, por lo tanto, puede ser unidimensional, es decir, un solo número real escalar.

* definir un método `forward`. Este método debe tomar nuestro texto y hacer lo siguiente:
  1. Aplicar la capa `Embeddings` a nuestras oraciones.
  2. Aplicar la capa `RNN` a los datos salidos de `Embeddings`
  3. Aplicar una capa densa `Linear` al estado oculto final de la `RNN`
  >NOTA: El estado oculto de RNN es de la forma: 
  `hidden = [ 1, batch size, hid dim ]`. Es posible que necesite aplicar un `squeeze` en la dimensión 0 para eliminar ese 1.


In [ ]:
#@markdown # Ejercicio 4



import torch.nn as nn

class Vanilla_RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
      # Inserte su código aquí.
 
        
    def forward(self, text):
      # Inserte su código aquí.


Ahora creamos una instancia de nuestra clase RNN.

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model1 = Vanilla_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

También creemos una función que nos diga cuántos parámetros entrenables tiene nuestro modelo para que podamos comparar el número de parámetros en diferentes modelos.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model1):,} parámetros entrenables')

El modelo tiene 1,988,005 parámetros entrenables


## Entrenar el modelo

Ahora vamos a configurar el entrenamiento y a entrenar el modelo.

Primero, crearemos un optimizador. Este es el algoritmo que usamos para actualizar los parámetros del módulo. Aquí, usaremos _descenso de gradiente estocástico_ (SGD). El primer argumento son los parámetros que serán actualizados por el optimizador y el segundo es la tasa de aprendizaje, es decir, cuánto cambiaremos los parámetros cuando hagamos una actualización de parámetros.

In [ ]:
import torch.optim as optim

optimizer1 = optim.Adam(model1.parameters(), lr=0.03)

A continuación, definiremos nuestra función de pérdida. 

La función de pérdida aquí es _entropía cruzada binaria con logits_.

Nuestro modelo genera actualmente un número real sin consolidar. Como nuestras etiquetas son 0 o 1, queremos restringir las predicciones a un número entre 0 y 1. Hacemos esto usando las funciones _sigmoidea_.

Luego usamos este escalar para calcular la pérdida usando entropía cruzada binaria.

La clase `BCEWithLogitsLoss` lleva a cabo los pasos de entropía cruzada sigmoidea y binaria.

In [ ]:
loss = nn.BCEWithLogitsLoss()

Usando `.to`, podemos ubicar al modelo y la función de pérdida en el GPU (si tenemos alguno). 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model1 = model1.to(device)
loss = loss.to(device)

Ya establecimos cómo la pérdida, sin embargo, tenemos que escribir nuestra función para calcular el accuracy.

Esta función primero alimenta las predicciones a través de una capa sigmoidea, reescalando los valores entre 0 y 1, luego los redondeamos al número entero más cercano. Esto redondea cualquier valor superior a 0,5 a 1 (un sentimiento positivo) y el resto a 0 (un sentimiento negativo).

Luego calculamos cuántas predicciones redondeadas son iguales a las etiquetas reales y la promediamos en todo el lote.

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    # aproximamos al entera más cercano
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convertimos a flotante para la división
    acc = correct.sum() / len(correct)
    return acc

La función `train_epoch` itera sobre todos los ejemplos, un lote a la vez.

Alimentamos el lote de oraciones, `text`, en el modelo. Es necesario usar la función `squeeze` ya que las predicciones tienen inicialmente la forma **[tamaño de lote, 1]**, y necesitamos eliminar la dimensión de tamaño 1 ya que PyTorch espera que la entrada de predicciones a nuestra función de pérdida tenga forma **[tamaño del lote]**.



In [ ]:
def train_epoch(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for label, text, _ in iterator:
        label = label.float().to(device)
        text = text.to(device)

        optimizer.zero_grad()
                
        predictions = model(text).squeeze(1)
        
        loss = criterion(predictions, label)
        
        acc = binary_accuracy(predictions, label)
        
        loss.backward()
        
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()


    return epoch_loss * batch_size / len(iterator), epoch_acc * batch_size / len(iterator)

`evaluate_epoch` es parecida a `train_epoch`, con algunas modificaciones ya que no desea actualizar los parámetros al evaluar.

In [ ]:
def evaluate_epoch(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for label, text, _ in iterator:
          
            label = label.float().to(device)
            text = text.to(device)

            predictions = model(text).squeeze(1)
            
            loss = criterion(predictions, label)
            
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss * batch_size/ len(iterator), epoch_acc * batch_size / len(iterator)

También vamos a crear una función para decirnos cuánto tarda una época en entrenarse para comparar los tiempos de entrenamiento entre modelos.

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Luego entrenamos el modelo a través de múltiples épocas, una época es un pase completo a través de todos los ejemplos en los conjuntos de entrenamiento y validación.

In [ ]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_epoch(model1, train_iter,
                                        optimizer1, loss, device)
    valid_loss, valid_acc = evaluate_epoch(model1, val_iter, loss, device)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 22s
	Train Loss: 0.824 | Train Acc: 49.93%
	 Val. Loss: 0.722 |  Val. Acc: 50.08%
Epoch: 02 | Epoch Time: 0m 21s
	Train Loss: 0.822 | Train Acc: 51.58%
	 Val. Loss: 0.749 |  Val. Acc: 51.56%
Epoch: 03 | Epoch Time: 0m 22s
	Train Loss: 0.707 | Train Acc: 56.49%
	 Val. Loss: 0.879 |  Val. Acc: 50.90%
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.760 | Train Acc: 54.89%
	 Val. Loss: 0.760 |  Val. Acc: 51.36%
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: nan | Train Acc: 0.21%
	 Val. Loss: nan |  Val. Acc: 0.00%


Aunque la perdida cae, la precisión es deficiente. Esto se debe a varios problemas con el modelo que mejoraremos a continuación.

Por último veamos la pérdida de nuestro modelo con el data ser de prueba.

In [ ]:
test_loss, test_acc = evaluate_epoch(model1, test_iter, loss, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: nan | Test Acc: 0.00%


# 2 - Análisis de Sentimiento Potenciado


## Preparando los datos

Usaremos **secuencias empaquetadas con padding**, lo que hará que nuestro RNN solo procese los tokens distintos a `<pad>` de nuestra secuencia, y para cualquier elemento `<pad>`, la "salida" será un tensor cero. Para usar secuencias empaquetadas con padding, tenemos que decirle al RNN la longitud de las secuencias reales. Afortunadamente, en la creación de nuestros `DataLoader` la función `collate` ya se encargaba de esto. La principal diferencia la tendremos en nuestro modelo, que deberá tomar recibir como entrada la longitud de la oración.

## Construcción del Modelo

###  Una arquitectura RNN diferente

En esta segunda parte, usaremos una arquitectura RNN diferente. Para ello usaremos podemos elegir entre una LSTM o una GRU. ¿Por qué estás son mejores que una RNN estándar? Los RNN estándar sufren el [problema del gradiente que se desvanece](https://en.wikipedia.org/wiki/Vanishing_gradient_problem). LSTM y GRU superan esto al usar de múltiples **_compuertas_** que controlan el flujo de información dentro y fuera de la memoria.

Recuerde que LSTM devuelve la celda de memoria y el estado oculto. Por lo cual debera manejar esa diferencia. Sin embargo, la predición del sentimiento todavía se realiza utilizando la variable oculta final, no el estado final de la celda, es decir, $ \hat {y} = f (h_T) $. Para ahorrarnos estos problemas, decidimos usar `GRU`

###  RNN Bidireccional

El concepto detrás de una RNN bidireccional es simple. Además de tener una RNN procesando las palabras en la oración desde la primera hasta la última (una RNN hacia adelante), tenemos una segunda RNN procesando las palabras en la oración desde **la última a la primera** (una RNN hacia atrás) . En el tiempo $ t $, la RNN hacia adelante está procesando la palabra $ x_t $, y la RNN hacia atrás está procesando la palabra $ x_{T-t + 1} $.

En PyTorch, los tensores de las variables ocultas (y los estados de celda) devueltos por las RNN hacia adelante y hacia atrás se apilan uno encima del otro en un solo tensor.

Hacemos nuestra predicción de sentimiento usando una concatenación del último estado oculto de la RNN hacia adelante (obtenido de la palabra final de la oración), $ h_T ^ \rightarrow $, y el último estado oculto de la RNN hacia atrás (obtenido a partir de la primera palabra de la oración), $ h_T ^ \leftarrow $, es decir, $ \hat {y} = f (h_T ^ \rightarrow, h_T ^ \leftarrow) $

La siguiente imagen muestra una RNN bidireccional, con la RNN hacia adelante en naranja, la RNN hacia atrás en verde y la capa lineal en plateado.

![](https://i.imgur.com/g1mAJXt.png)

### RNN Multicapa

Las RNN multicapa (también llamados *RNN profundas*) son otro concepto simple. La idea es que agreguemos RNN adicionales sobre la RNN estándar inicial, donde cada RNN agregado es otra *capa*. La salida de la variable oculta de la primera RNN (inferior) en el tiempo $ t $ será la entrada de la RNN por encima de ella en el paso de tiempo $ t $. Luego, la predicción se realiza a partir de la variable oculta final de la capa final (más alta).

La siguiente imagen muestra una RNN unidireccional multicapa, donde el número de capa se da como superíndice. También tenga en cuenta que cada capa necesita su propia variable oculta inicial, $ h_0 ^ L $.

![](https://i.imgur.com/wBKSBhx.png)

### Regularización

Aunque hemos agregado mejoras a nuestro modelo, cada una de ellas agrega parámetros adicionales. Cuantos más parámetros tenga en su modelo, mayor será la probabilidad de que su modelo se sobreajuste (memorice los datos de entrenamiento, lo que provoca un error de entrenamiento bajo pero un error de validación/prueba alto, es decir, una generalización deficiente para ejemplos nuevos e inéditos). 

Para combatir esto, usamos la regularización. Más específicamente, usamos un método de regularización llamado **dropout**. El dropout funciona al *eliminar* (configurando en 0) neuronas en una capa al azar durante un pase hacia adelante. La probabilidad de que se descarte cada neurona se establece mediante un hiperparámetro y cada neurona con dropout aplicado se considera de forma independiente. Una teoría sobre por qué funciona el dropout es que un modelo con parámetros abandonados puede verse como un modelo "más simple" (menos parámetros). Las predicciones de todos estos modelos "más simples" (una para cada pase hacia adelante) se promedian juntas dentro de los parámetros del modelo. Por lo tanto, su único modelo puede considerarse como un conjunto de modelos más simples, ninguno de los cuales está sobre parametrizado y, por lo tanto, no debe sobreajustarse.

### Detalles de Implementación

Otra adición a este modelo es que no vamos a aprender el embedding del token `<pad>`. Esto se debe a que queremos decirle explícitamente a nuestro modelo que los tokens de relleno son irrelevantes para determinar el sentimiento de una oración. Esto significa que el embedding del token de padding permanecerá como se inicializó (todos ceros). Hacemos esto pasando el índice de nuestro token `<pad>` como el argumento `padding_idx` a la capa` nn.Embedding`.

Para usar un GRU en lugar del RNN estándar, usamos `nn.GRU` en lugar de` nn.RNN`. Además, tenga en cuenta que el LSTM devuelve la "salida" y una tupla con el estado final de la variable oculta y de la "celda", mientras que el RNN estándar solo devuelve la "salida" y el estado final de la variable oculta.

Como el estado oculto final de nuestro GRU tiene un componente hacia adelante y hacia atrás, que se concatenarán juntos, el tamaño de la entrada a la capa `nn.Linear` es el doble que el tamaño de la dimensión oculta.

La implementación de la bidireccionalidad y la adición de capas adicionales se realizan pasando valores para los argumentos `num_layers` y` bidirectional` para el LSTM.

El dropout se implementa inicializando una capa `nn.Dropout` (el argumento es la probabilidad de descartar cada neurona) y usándola dentro del método` forward` después de cada capa a la que queremos aplicar el dropout. **Nota**: nunca use dropout en las capas de entrada o salida (`text` o` fc` en este caso), solo desea usarlo en las capas intermedias. Las LSTM tienes un argumento de "dropout" que añade dropout en las conexiones entre las variables ocultas en una capa y las variables ocultas en la siguiente capa.

Como estamos pasando las longitudes de nuestras oraciones para poder usar secuencias empaquetadas con padding, tenemos que agregar un segundo argumento, `text_lengths`, a` forward`.

Antes de pasar nuestras embeddings al RNN, necesitamos empaquetarlas, lo que hacemos con `nn.utils.rnn.packed_padded_sequence`. Esto hará que nuestro RNN solo procese los elementos no rellenados de nuestra secuencia. Entonces, el RNN devolverá `package_output` (una secuencia empaquetada) así como los estados` hidden` y `cell` (ambos son tensores). Sin secuencias empaquetadas con padding, ` hidden` es un tensor del último elemento de la secuencia, que probablemente será un token de relleno; sin embargo, cuando se utilizan secuencias empaquetadas con padding, ambos son del último elemento no rellenado de la secuencia. Tenga en cuenta que el argumento `lengths` de` package_padded_sequence` debe ser un tensor de CPU, por lo que lo convertimos explícitamente en uno usando `.to('cpu')`.

Luego descomprimimos la secuencia de salida, con `nn.utils.rnn.pad_packed_sequence`, para transformarla de una secuencia empaquetada a un tensor. Los elementos de "salida" de los tokens de relleno serán tensores cero (tensores donde cada elemento es cero). 

El estado final de la variable oculta, `hidden`, tiene una forma de **[núm capas * núm direcciones, tamaño de lote, dimensión de la variable oculta]**. Estos están ordenados: **[forward_layer_0, backward_layer_0, forward_layer_1, backward_layer 1, ..., forward_layer_n, backward_layer n]**. Queremos todas las variables ocultas en su versión hacia adelante y hacia atras para alimentar a la capa final. Para esto debemos usar la función `chunk` para recuparar cada variable oculta y luego `cat` para transformalas en un matriz.



### Implementación de nuestro modelo

A continuación, deberá crear la clase `RNN`.

Para ello tenga encuenta que debera:

* definir un constructor, `__init__` con tres capas:
  * Una instancia de `Embedding`
  * Una instancia de `GRU`
  * Una instancia de `Linear`

* Recuerde también que:
  * La dimensión de entrada de nuestra red es la dimensión de los vectores one-hot. Es decir, es el tamaño del vocabulario.
  * La dimensión a la salida de `Embedding` es el tamaño de los vectores de palabras densas. Suele tener entre 50 y 250 dimensiones, pero depende del tamaño del vocabulario.
  * La instancia de `Embebbing` debe recibir 1 argumentos adicionales: `padding_idx`. Este argumento le indica al embedding que debe ignorar los rellenos.
  * La dimensión del estado oculto es el tamaño de las variables ocultas. Suele rondar entre 100 y 500 dimensiones, pero también depende de factores como el tamaño del vocabulario, el tamaño de los vectores densos y la complejidad de la tarea.
  * La instancia de GRU debe recibir 3 argumentos adicionales: `dropout`, `bidirectional`, `num_layers`
  * La dimensión de salida suele ser el número de clases, sin embargo, en el caso de solo 2 clases, el valor de salida está entre 0 y 1 y, por lo tanto, puede ser unidimensional, es decir, un solo número real escalar.
  * Como queremos las variables ocultas hacia adelante y hacia atras, debemos decirle a la capa lineal final que recibira un tensor de la forma `hidden_dim * num_layers * 2` para redes bidireccionales o `hidden_dim * num_layers` para redes no bidireccionales

* definir un método `foward`. Este método tiene como entrada nuestro texto `text`, la longitud de nuestras oraciones `text_lengths`. Con esas entradas debe hacer lo siguiente:
  1. Aplicar la capa `Embeddings` a nuestras oraciones, usando el argumento `padding_idx`
  2. Aplicar un dropout la salida de la capa `Embeddings`
  3. Ahora debemos empaquetar nuestra salida con `padding`. Para ello usaremos la función  `pack_padded_sequence` en el modulo `torch.nn.utils.rnn`. A este método debemos pasarle nuestra salida de `Embedding` con el dropout y la longitud del texto, `text_lengths`. Ademas necesitamos que `text_lengths`este en CPU.
  2. Aplicar la capa `GRU` a los datos salidos de `Embeddings`, usando el argumento `dropout`
  4. El argumento dropout de `GRU` aplica dropout a cada estado oculto, excepto al último. Por esto debemos agregar un dropout a la última variable oculta de `GRU`
  5. Además debemos reestructurar nuestra salida de GRU. Tenemos que aplicar las funciones `chunk`,  `cat` y `squeeze` del modulo `torch` para pasar de  de un tensor de la forma:

    `hidden = [num layers * num directions, batch size, hid dim]`

    a otro de la forma 

    `hidden = [batch size, hid dim * num layers * num direction]`
    
    donde `num directions` es 2 para RNN bidireccionales o 1 para RNN convencionales.
  3. Aplicar una capa densa `Linear` al estado oculto final de la `GRU`

> NOTA: A la salida de GRU tenemos un tensor `output` que ha sido empaquetado. En este problema no necesitamos usar el tensor `output`, pero en otros problemas, ese tensor empaqutado puede ser util. Para desempaquetarlo existe la función `pad_packed_sequence(packed_output)` que devuelve las salidas desempaquetadas con su longitud.

In [ ]:
#@markdown # Ejercicio 5

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, pad_index,
                 hidden_dim, output_dim, num_layers, bidirectional, dropout):
      # Inserte su código aquí.

        
    def forward(self, text, text_lengths):
      # Inserte su código aquí.

Ahora creamos una instancia de nuestra clase RNN.

La dimensión de entrada es la dimensión de los vectores one-hot, que es igual al tamaño del vocabulario.

La dimensión de embedding es el tamaño de los vectores de palabras densas. Suele tener entre 50 y 250 dimensiones, pero depende del tamaño del vocabulario.

Como ya hemos creado un vocabulario, debemos informale a la red cual es nuestro token de relleno

La dimensión oculta es el tamaño de las variables ocultas. Suele rondar entre 100 y 500 dimensiones, pero también depende de factores como el tamaño del vocabulario, el tamaño de los vectores densos y la complejidad de la tarea.

Para activar la bidirecionalidad de una RNN en `torch` debemos pasar una variable `True` para que que el *framework* habilite la bidireccionalidad.

Analogamente, debemos pasar el número de capas RNN, en este caso solo usaremos 2

La dimensión de salida suele ser el número de clases, sin embargo, en el caso de solo 2 clases, el valor de salida está entre 0 y 1 y, por lo tanto, puede ser unidimensional, es decir, un solo número real escalar.

Por último debemos decir cuanto dropout agregaremos

In [ ]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
PAD_IDX = vocab['<PAD>']
HIDDEN_DIM = 256
NUM_LAYERS = 2
BIDIRECTIONAL = True
OUTPUT_DIM = 1
DROPOUT = 0.5

model2 = RNN(INPUT_DIM, EMBEDDING_DIM, PAD_IDX,
            HIDDEN_DIM,OUTPUT_DIM, NUM_LAYERS, BIDIRECTIONAL, DROPOUT)

También creemos una función que nos diga cuántos parámetros entrenables tiene nuestro modelo para que podamos comparar el número de parámetros en diferentes modelos.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'El modelo tiene {count_parameters(model1):,} parámetros entrenables')

El modelo tiene 1,988,005 parámetros entrenables


## Entrenar el modelo

Reusaremos la funciones anteiores. La única diferencia es que debemos crear una nueva instancia de optimizador que aplique sobre nuestro nuevo modelo.

In [ ]:
import torch.optim as optim

optimizer2 = optim.Adam(model2.parameters())

Además, debemos volver a a enviar nuestro modelo a la GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model2 = model2.to(device)
loss = loss.to(device)

Como nuestro nuevo modelo requiere de las longitudes de de las oraciones, debemos modificar la función de entrenamiento y de evaluación.

In [ ]:
def train_epoch_wl(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    stop = 0
    for label, text, length in iterator:
        label = label.float().to(device)
        text = text.to(device)

        optimizer.zero_grad()
                
        predictions = model(text, length).squeeze(1)
        
        loss = criterion(predictions, label)
        
        acc = binary_accuracy(predictions, label)
        
        loss.backward()
        
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()


    return epoch_loss * batch_size / len(iterator), epoch_acc * batch_size / len(iterator) 

Hacemos lo mismo con la otra función.

In [ ]:
def evaluate_epoch_wl(model, iterator, criterion, device):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        stop = 0
        for label, text, length in iterator:
          

            label = label.float().to(device)
            text = text.to(device)

            predictions = model(text, length).squeeze(1)
            
            loss = criterion(predictions, label)
            
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss * batch_size / len(iterator), epoch_acc * batch_size / len(iterator) 

Luego entrenamos el modelo a través de múltiples épocas, una época es un pase completo a través de todos los ejemplos en los conjuntos de entrenamiento y validación.

En cada época, si la pérdida de validación es la mejor que hemos visto hasta ahora, guardaremos los parámetros del modelo y luego, una vez finalizado el entrenamiento, usaremos ese modelo en el conjunto de prueba.

In [ ]:
N_EPOCHS = 10

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_epoch_wl(model2, train_iter,
                                        optimizer2, loss, device)
    valid_loss, valid_acc = evaluate_epoch_wl(model2, val_iter, loss, device)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 40s
	Train Loss: 0.676 | Train Acc: 57.85%
	 Val. Loss: 0.664 |  Val. Acc: 60.88%
Epoch: 02 | Epoch Time: 0m 41s
	Train Loss: 0.613 | Train Acc: 66.19%
	 Val. Loss: 0.534 |  Val. Acc: 74.20%
Epoch: 03 | Epoch Time: 0m 40s
	Train Loss: 0.480 | Train Acc: 77.32%
	 Val. Loss: 0.504 |  Val. Acc: 80.96%
Epoch: 04 | Epoch Time: 0m 40s
	Train Loss: 0.381 | Train Acc: 83.47%
	 Val. Loss: 0.397 |  Val. Acc: 84.20%
Epoch: 05 | Epoch Time: 0m 40s
	Train Loss: 0.323 | Train Acc: 86.62%
	 Val. Loss: 0.365 |  Val. Acc: 86.14%


In [ ]:
test_loss, test_acc = evaluate_epoch_wl(model2, test_iter, loss, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.368 | Test Acc: 85.46%


## Entrada del usuario

Ahora podemos usar nuestro modelo para predecir el sentimiento de cualquier oración que le demos. Como ha sido entrenado en reseñas de películas, las oraciones proporcionadas también deben ser reseñas de películas.

Cuando se usa un modelo para la predicción, siempre debe estar en modo de evaluación. Si se sigue este tutorial paso a paso, entonces ya debería estar en modo de evaluación (desde que ejecutamos `evaluate()` en el conjunto de prueba), sin embargo, lo configuramos explícitamente para evitar cualquier riesgo.

Nuestra función `predict_sentiment` debe hacer algunas cosas:
- configurar el modelo en modo de evaluación
- tokenizar la oración, es decir, pasar de una cadena sin procesar a una lista de tokens
- indexar los tokens convirtiéndolos en su representación entera de nuestro vocabulario
- obtener la longitud de nuestra secuencia
- convertir los índices, que son una lista de Python en un tensor de PyTorch
- agregar una dimensión de lote al hacer `unsqueeze`
- convertir la longitud de la secuencia en un tensor
- rescalar la predicción de salida a un rango entre 0 y 1 con la función "sigmoide"
- convertir el tensor que contiene un valor único en un número entero con el método `item()`

Esperamos que las reseñas con un sentimiento negativo devuelvan un valor cercano a 0 y que las reseñas positivas devuelvan un valor cercano a 1.

In [ ]:
# @markdown # Ejercicio 6


def predict_sentiment(model, sentence):
  # inserte su código aquí

Un ejemplo de reseña negativa...

In [ ]:
predict_sentiment(model2, "This movie is terrible",)

0.014458732679486275

Un ejemplo de reseña positiva...

In [ ]:
predict_sentiment(model2, "Greatest film ever!")


0.7371154427528381